# 2D CODE

In [1]:
import numpy as np
import matplotlib
import math
import time
import pickle


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import itertools


import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import tensorflow_probability as tfp


%matplotlib inline

## Run auxiliary functions:

In [2]:
%run ./auxiliary_func_noprint.ipynb

## Choose input/output dimensions and look at the examples

In [3]:
## Allowed input/output values in this preliminary version are (ni=2 no=1) (ni=3 no=1) (ni=3 no=2) and (ni=1 no=1) 

ni=2 #number of input variables  
no=1 #number of output solutions 

#LOOK AT THE PDE DICTIONARIES MATCHING THE DIMENSION REQUIREMENTS
#################################################################################
### outputfunction is named u, v
### first derivatives are named dudt, dudx, dudy
### pure second derivatives are named dduddt, dduddx, dduddy
### mixed second derivative (as ddudtdx) need to be coded in the loss function, we will provide an automatic derivation in future updates.
############## NOTATION #####################
## eomdict: impose PDE constraints. The form needs to be '[ eq1 , eq2 ]' 
## ICdict: Boundary conditions on t belonging to [t0,tL].
##         The form needs to be [ '[ u - u(t0,x,y),dudt - dudt(t0,x,y)]' , '[u - u(tL,x,y),dudt - dudt(tL,x,y)]' ] 
##         If no condition needs to be provided on some boundary just replace (u - u(t,x,y0) ) with  [none].
## boardx: Boundary conditions on x belonging to [x0,xL].
##         The form needs to be [ '[u - u(t,x0,y),dudx - dudx(t,x0,y)]' , '[u - u(t,xL,y),dudx - dudx(t,xL,y)]' ]   
##         If no condition needs to be provided on some boundary just replace (u - u(t,x,y0) ) with  [none].
## boardy: Boundary conditions in y belonging to [y0,yL].
##         The form needs to be [ '[u - u(t,x,y0),dudy - dudy(t,x,y0)]' , '[u - u(t,x,yL),dudy - dudy(t,x,yL)]' ] 
##         If no condition needs to be provided on some boundary just write  [none].
## wdict:  weights to be used in loss function
##         The form needs to be [w_bulk,w_IC,w_board]

eqname_dict=eqname_gen(ni,no)
analytic=eqname_analytic(ni,no)
sol=sol_analytic(ni,no)

if ni==1:
    eomdict,ICdict=dictgen(ni,no)
elif ni==2:
    eomdict,ICdict,boardx,wdict=dictgen(ni,no)
elif ni==3:
    eomdict,ICdict,boardx,boardy,wdict=dictgen(ni,no)

eomdict

{'wave': '[dduddt-1.**2*dduddx]',
 'waveN': '[dduddt-1.**2*dduddx]',
 'twave': '[dudt-1.**2*dudx]',
 'heat': '[dudt-0.05*dduddx]',
 'heat2f': '[dudt-0.01*dduddx]',
 'heat0': '[dudt-0.05*dduddx]',
 'poisson': '[dduddt+dduddx + 2. * np.pi**2. * tf.sin(np.pi * t) * tf.sin(np.pi * x)]',
 'poisson1': '[dduddt+dduddx-10.*(t-1)*tf.math.cos(5.*x)+25.*(t-1)*(x-1)*tf.math.sin(5.*x)]',
 'AdvectionDiffusion': '[dudt - 0.25 * dduddx]',
 'Burgers': '[dudt + u * dudx - 0.25 * dduddx]',
 'poisson_par': '[(dduddt+dduddx-tf.exp(-(t**2+10.*x**2)))]',
 'par': '[(dduddt+dduddx-4.)]'}

In [4]:
#####################################################
#### Choose the equation of motion to be studied
#####################################################
### 3to1 D equations studied: ['waveb','wave','wavet', 'poisson1', 'poisson2', 'poisson3','heat1','heat2']
### 3to2 D equations studied: ['GT','LO','NS']
### 2to1 D equations studied: [{'wave','waveN','twave','heat','heat2f','heat0','poisson','poisson1','AdvectionDiffusion','Burgers','poisson_par','par']
### 1to1 D equations studied: [{'oscillon', 'mat', 'exp', 'wave', 'dho', 'linear', 'delay', 'gaussian', 'stiff', 'twofreq'}]
#####
##Create the list of coordinate boundaries, i.e. boundaries=[t_0,t_max,x_0,x_max,y_0,y_max]
#### Boudaries used in the paper
#bd=[0.,1.,0.,1.,0.,1.]               # 3to1 D
#bd=[0.,1.,0.,2.*np.pi,0.,2.*np.pi]   #'GT'
#bd=[0.1,1.,-2.,2.,-2.,2.]            #'LO'
#bd=[0.,10.,0.,1.,0.,1.]              #'NS'
#bd=[0.,1.,0.,1.]                     # 2to1 D
#bd=[0.,20.]                          # 1to1 D

bd=[0.,1.,0.,1.] 



## define number of points to be sampled in the bulk, the IC and on each boundary surface
## (for the 1D case (n_IC,n_board) will effectively evaluate to (1,0), irrespective of the chosen values)
n_bulk=1000
n_IC=200
n_board=200


##Initialise the network
n_l=10 #number of neurons per branch

#create base frequencies to initialize the network
pi=tf.constant(np.pi,dtype=tf.float32)
none=tf.constant(0.,dtype=tf.float32)

freq_t,freq_x=base_freq(ni,bd)

step=1.
model=dNNsolve(ni,no,n_l,bd,step)

model.save_weights(f"model_seq_{ni}D_{n_l}nodes.h5")


## Training and plots

In [ ]:
epochs_adam=400  
batch_sz=256

f= open("2to1_results.txt","w+")
twoD_dict={'disk':['par','poisson_par'],
           'rect':['wave','waveN','twave','heat','heat2f','heat0','poisson1','poisson','AdvectionDiffusion','Burgers']}
for domain in twoD_dict:
    for eom in twoD_dict[domain]:


        #Define dataset from random sampling the domain
        
        if domain=='rect':
            X_data, t, x=  (random_sampling(ni,domain))(bd,n_bulk,n_IC,n_board)
            #Define IC, boundary and bulk counters (I_t0,I_tL,I_x0,I_xL,I_y0,I_yL,I_bulk)
            I=counters(ni,'rect')(bd,t,x)
            fake_output=tf.concat([X_data,I],axis=1)
            inputs=[t, x]
        if domain=='disk':
            X_data, t, x=  (random_sampling(ni,domain))(bd,n_bulk,n_board)
            #Define IC, boundary and bulk counters (I_t0,I_tL,I_x0,I_xL,I_y0,I_yL,I_bulk)
            I=counters(ni,'disk')(bd,t,x)
            fake_output=tf.concat([X_data,I],axis=1)
            inputs=[t, x]
        
        ########################
        start=time.time() 
        #initialization of models
        model.load_weights(f"model_seq_{ni}D_{n_l}nodes.h5") #Restore weights to original values at each iteration
        loss_fun=to_loss_2to1(eom,domain)

        #Initialise BFGS method and compile the model
        func = function_factory(model, loss_fun, fake_output)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                               loss=loss_fun,run_eagerly=False)

        #Training with Adam
        hist = model.fit(x=inputs, y=fake_output, batch_size = batch_sz, epochs=epochs_adam, verbose=0, callbacks = 
                           [tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', mode='min', factor=1./2., patience=30, min_lr=1e-4)])


        #Training with BFGS
        init_params = tf.dynamic_stitch(func.idx, model.trainable_variables)
        results = tfp.optimizer.bfgs_minimize(value_and_gradients_function=func, initial_position=init_params,tolerance=1e-20, max_iterations=5000)    
        func.assign_new_model_parameters(results.position)



        #Store loss data
        loss_adam=np.array(hist.history['loss'],dtype=np.float32) 
        history_lbfgs=np.array(func.history)
        hist_lbfgs=np.array(history_lbfgs[:,0])
        loss=np.concatenate((loss_adam,hist_lbfgs))
        end=time.time()

        if domain=='rect':
            n_plt=51
            pred, smse, tplt, xplt=points_plt_mse(ni,no,bd,model,sol,domain='rect')

        elif domain=='disk':
            n_plt=51
            pred, smse, tplt, xplt=points_plt_mse(ni,no,bd,model,sol,domain='disk')

        eps=1e-20 ##small parameter to avoid inf values
        f.write('\n EOM: ' + eom)
        f.write('\n Neurons per branch:  %s' % (n_l))
        f.write('\n Epochs:  %s' % (loss.size))
        f.write('\n Total Time: %.4f' % (end-start))
        f.write('\n Final log10(Loss Adam): %.4f' % (np.log10(loss_adam[-1])+eps))
        f.write('\n Final log10(Loss): %.4f' % (np.log10(loss[-1])+eps))
        f.write('\n Log10 of  Sqrt Mean squared error:  %.4f' % (np.log10(smse+1e-20)+eps))
        f.write('\n log10(Loss bulk): %.4f log10(Loss IC): %.4f log10(Loss board): %.4f' % (np.log10(eps+np.array(history_lbfgs[:,1])[-1]),np.log10(eps+np.array(history_lbfgs[:,2])[-1]),np.log10(eps+np.array(history_lbfgs[:,3])[-1])))
        f.write('\n #######################################################')

f.close()